<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/%E3%80%8C%E6%95%B4%E5%90%88%E7%89%88FED%E5%8D%87%E9%99%8D%E6%81%AF%E6%B1%BA%E7%AD%96%E6%A8%A1%E5%9E%8B_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
"""
强化版联准会利率预测系统 (错误修正版)
1. 验证最新指标代码
2. 三重数据备援机制
3. 智能异常处理
更新日期：2024年1月
"""
!pip install fredapi tenacity
from fredapi import Fred
import pandas as pd
import logging
from tenacity import retry, stop_after_attempt, wait_exponential

# ===== 已验证的最新指标配置 =====
API_KEY = 'bb53d6e4600f49124c427b8b66b35d81'
指標配置 = {
    '預期通膨率': {
        '主要代碼': 'MICH',  # 密西根大学1年通胀预期
        '備援路徑': [
            ('T10YIE', '市場10年通膨預期'),  # 基於國債損益平衡率
            ('CPIAUCSL', '實際CPI年增率')    # 歷史實際數據
        ],
        '單位': '%',
        '合理範圍': (0, 20)  # 新增數據驗證
    },
    '預期失業率': {
        '主要代碼': 'SPFURTCT',  # 費城聯儲正確代碼 (10年失业率预测中位数)
        '備援路徑': [
            ('CIVPART', '勞動參與率'),       # 替代性勞動指標
            ('UNRATE', '官方失業率'),        # 實際失業數據
            ('U6RATE', '廣義失業率')        # 包含就業不足
        ],
        '單位': '%',
        '合理範圍': (2, 30)
    },
    '當前利率': {
        '主要代碼': 'DFF',          # 聯邦基金有效利率
        '備援路徑': [
            ('FEDFUNDS', '歷史平均利率'),
            ('IORB', '準備金利率')  # 新增備援
        ],
        '單位': '%',
        '合理範圍': (0, 25)
    }
}

# ===== 智能数据抓取引擎 =====
class 智能數據抓取器:
    def __init__(self, api_key):
        self.fred = Fred(api_key=api_key)
        self.日誌器 = logging.getLogger('FRED引擎')
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )

    def _數據驗證(self, 數值, 配置):
        """確保數據在合理範圍內"""
        最小值, 最大值 = 配置['合理範圍']
        if not (最小值 <= 數值 <= 最大值):
            raise ValueError(f"數據越界: {數值}% 超出{最小值}-{最大值}%範圍")
        return 數值

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def _安全獲取(self, 代碼):
        """帶自動重試和驗證的數據層"""
        try:
            原始數據 = self.fred.get_series(代碼)
            if 原始數據.empty:
                raise ValueError("空數據集")
            最新值 = 原始數據.iloc[-1]
            self.日誌器.info(f"成功獲取: {代碼}")
            return 最新值
        except Exception as 異常:
            self.日誌器.warning(f"數據異常 {代碼}: {str(異常)}")
            return None

    def 獲取經濟指標(self, 指標名稱):
        """多層次數據獲取管道"""
        配置 = 指標配置[指標名稱]

        # 第一層：主要指標
        主要代碼 = 配置['主要代碼']
        數值 = self._安全獲取(主要代碼)
        if 數值 is not None:
            try:
                return self._數據驗證(數值, 配置)
            except Exception as 驗證異常:
                self.日誌器.error(f"數據驗證失敗 {主要代碼}: {str(驗證異常)}")

        # 第二層：備援指標
        for 代碼, 說明 in 配置['備援路徑']:
            self.日誌器.info(f"啟用備援: {說明}({代碼})")
            備援值 = self._安全獲取(代碼)
            if 備援值 is not None:
                try:
                    return self._數據驗證(備援值, 配置)
                except Exception as 驗證異常:
                    self.日誌器.warning(f"備援數據異常 {代碼}: {str(驗證異常)}")
                    continue

        # 第三層：緊急替代值
        self.日誌器.critical(f"所有來源失效: {指標名稱}")
        return self._取得歷史平均(配置['主要代碼'])

    def _取得歷史平均(self, 代碼):
        """最終保障：歷史3年平均"""
        try:
            歷史數據 = self.fred.get_series(代碼, observation_start='2019-01-01')
            return 歷史數據.mean() if not 歷史數據.empty else None
        except:
            return None

# ===== 利率決策模型強化版 =====
class 聯準會決策引擎:
    def __init__(self):
        # 泰勒規則參數庫
        self.參數庫 = {
            '標準版': {
                '目標通膨': 2.0,
                '自然失業率': 4.0,
                '通膨係數': 0.5,
                '失業係數': -1.0,
                '中性利率': 2.5
            },
            '鷹派版': {
                '目標通膨': 2.0,
                '自然失業率': 3.8,
                '通膨係數': 0.7,
                '失業係數': -0.8,
                '中性利率': 3.0
            }
        }
        self.當前模式 = '標準版'
        self.最大單次調整 = 0.5  # 50基點

    def 切換模式(self, 模式名稱):
        """動態調整政策取向"""
        if 模式名稱 in self.參數庫:
            self.當前模式 = 模式名稱
            self.日誌器.info(f"已切換至{模式名稱}模式")
        else:
            raise ValueError(f"無效模式: {模式名稱}")

    def 生成決策(self, 當前利率, 通膨率, 失業率):
        """核心決策算法"""
        # 參數載入
        參數 = self.參數庫[self.當前模式]

        # 缺口計算
        通膨缺口 = 通膨率 - 參數['目標通膨']
        失業缺口 = 失業率 - 參數['自然失業率']

        # 泰勒規則計算
        目標利率 = 參數['中性利率'] + \
                (參數['通膨係數'] * 通膨缺口) + \
                (參數['失業係數'] * 失業缺口)

        # 利率調整限制
        調整幅度 = max(min(目標利率 - 當前利率, self.最大單次調整), -self.最大單次調整)
        新利率 = max(當前利率 + 調整幅度, 0)  # 零利率下限

        # 生成決策描述
        基點變化 = int(round(調整幅度 * 100, 0))
        if 基點變化 > 0:
            決策 = f"升息{abs(基點變化)}基點"
        elif 基點變化 < 0:
            決策 = f"降息{abs(基點變化)}基點"
        else:
            決策 = "維持利率不變"

        return {
            '政策決策': 決策,
            '新利率': 新利率,
            '調整幅度': 調整幅度,
            '理論利率': 目標利率,
            '使用模式': self.當前模式
        }

# ===== 系統執行 =====
if __name__ == '__main__':
    # 初始化組件
    數據引擎 = 智能數據抓取器(API_KEY)
    決策系統 = 聯準會決策引擎()

    try:
        # 獲取即時數據
        經濟指標 = {
            指標: 數據引擎.獲取經濟指標(指標)
            for 指標 in 指標配置
        }

        # 執行決策分析
        決策結果 = 決策系統.生成決策(
            當前利率=經濟指標['當前利率'],
            通膨率=經濟指標['預期通膨率'],
            失業率=經濟指標['預期失業率']
        )

        # 專業報告輸出
        print("\n=== 聯準會利率決策智能報告 ===")
        print(f"當前基準利率: {經濟指標['當前利率']:.2f}%")
        print(f"通膨預期指標: {經濟指標['預期通膨率']:.2f}%")
        print(f"勞動市場指標: {經濟指標['預期失業率']:.2f}%")
        print("----------------------------------")
        print(f"政策取向模式: {決策結果['使用模式']}")
        print(f"理論最適利率: {決策結果['理論利率']:.2f}%")
        print(f"決策建議: {決策結果['政策決策']}")
        print(f"預測新利率: {決策結果['新利率']:.2f}%")
        print("\n※包含三重數據保障機制")

    except Exception as 異常:
        print(f"\n系統錯誤: {str(異常)}")
        print("建議處理步驟:")
        print("1. 檢查FRED指標代碼有效性")
        print("2. 確認網路連線狀態")
        print("3. 驗證API金鑰權限")


=== 聯準會利率決策智能報告 ===
當前基準利率: 4.33%
通膨預期指標: 4.30%
勞動市場指標: 4.20%
----------------------------------
政策取向模式: 標準版
理論最適利率: 3.45%
決策建議: 降息50基點
預測新利率: 3.83%

※包含三重數據保障機制
